In [3]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

# api_file ="__api_file_path__"
with open(api_file, 'r') as f:
    openai.api_key = f.read().strip()

### Reference
Full detailed information about fine-tuning can be found in the OpenAI cookbook for ["How to fine-tune chat models"](https://cookbook.openai.com/examples/how_to_finetune_chat_models).  
This notebook adapts code from the above cookbook and applied it to a new project.

### Fine-tuning project: formatted output from news headlines
We demonstrate fine tuning with news headline data from [Kaggle](https://www.kaggle.com/datasets/rmisra/news-category-dataset/)

- Misra, Rishabh. "News Category Dataset." arXiv preprint arXiv:2209.11429 (2022).
- Misra, Rishabh and Jigyasa Grover. "Sculpting Data for ML: The first act of Machine Learning." ISBN 9798585463570 (2021).

For demonstration purpose, we reduce the data sample size by taking out the first 100 entries in each category of business, politics, travel, sports and wellness. We then shuffle the entries to create the news.csv file for this notebook.

In [4]:
filename = 'news.csv'
news_df = pd.read_csv(filename)
news_df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffingtonpost.com/entry/swoon-the...,*Swoon*: The Dreamy Italian Villa From ‘Call M...,TRAVEL,We’re calling our broker ASAP.,"PureWow, Editorial Partner",2018-02-27
1,https://www.huffingtonpost.com/entry/errol-mus...,Elon Musk's Estranged Father Has Child With St...,BUSINESS,The young woman was only 4 years old when Erro...,Jenna Amatulli,2018-03-26
2,https://www.huffingtonpost.com/entry/toys-r-us...,This Sad Version Of The Toys R Us Jingle Will ...,BUSINESS,Sniff.,Ed Mazza,2018-03-16
3,https://www.huffpost.com/entry/biden-climate-e...,Biden Takes Modest Executive Action After Clim...,POLITICS,The executive push includes $2.3 billion in cl...,Chris D'Angelo,2022-07-20
4,https://www.huffingtonpost.com/entry/lonely-pl...,Lonely Planet's Top European Destinations Of 2...,TRAVEL,These underrated travel destinations in Europe...,Brittany Nims,2018-05-22


#### Data preparation
- Create helper function for composing messages

In [5]:
# Create functions for generating messages
system_message = "You are a expert news editor. You give author name and sort news headline into 5 categories: BUSINESS, POLITICS, TRAVEL, SPORTS, WELLNESS."

def create_user_message(row):
    return f"""Headline: {row['headline']}\n\n Author: {row['authors']}\n\n"""

def create_assistant_message(row):
    return f"This is a {row['category']} piece by {row['authors']}" 

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    assist_message = create_assistant_message(row)
    messages.append({"role": "assistant", "content": assist_message}) 

    return {"messages": messages}

pprint(prepare_example_conversation(news_df.iloc[0]))
# pprint(prepare_example_conversation(news_df.iloc[1]))

{'messages': [{'content': 'You are a expert news editor. You give author name '
                          'and sort news headline into 5 categories: BUSINESS, '
                          'POLITICS, TRAVEL, SPORTS, WELLNESS.',
               'role': 'system'},
              {'content': 'Headline: *Swoon*: The Dreamy Italian Villa From '
                          '‘Call Me By Your Name’ Is For Sale\n'
                          '\n'
                          ' Author: PureWow, Editorial Partner\n'
                          '\n',
               'role': 'user'},
              {'content': 'This is a TRAVEL piece by PureWow, Editorial '
                          'Partner',
               'role': 'assistant'}]}


- Create training and validation data set

In [30]:
# use the first 21 rows of the dataset for training
training_df = news_df.loc[0:20]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:2]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a expert news editor. You give author name and sort news headline into 5 categories: BUSINESS, POLITICS, TRAVEL, SPORTS, WELLNESS.'}, {'role': 'user', 'content': 'Headline: *Swoon*: The Dreamy Italian Villa From ‘Call Me By Your Name’ Is For Sale\n\n Author: PureWow, Editorial Partner\n\n'}, {'role': 'assistant', 'content': 'This is a TRAVEL piece by PureWow, Editorial Partner'}]}
{'messages': [{'role': 'system', 'content': 'You are a expert news editor. You give author name and sort news headline into 5 categories: BUSINESS, POLITICS, TRAVEL, SPORTS, WELLNESS.'}, {'role': 'user', 'content': "Headline: Elon Musk's Estranged Father Has Child With Stepdaughter, Says It's 'God's Plan'\n\n Author: Jenna Amatulli\n\n"}, {'role': 'assistant', 'content': 'This is a BUSINESS piece by Jenna Amatulli'}]}


In [31]:
validation_df = news_df.loc[21:30]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

- Write training and validation data to the required jsonl format

In [32]:
# write out data in jsonl format
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

training_file_name = "tmp_news_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_news_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [33]:
!head -n 2 tmp_news_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a expert news editor. You give author name and sort news headline into 5 categories: BUSINESS, POLITICS, TRAVEL, SPORTS, WELLNESS."}, {"role": "user", "content": "Headline: *Swoon*: The Dreamy Italian Villa From \u2018Call Me By Your Name\u2019 Is For Sale\n\n Author: PureWow, Editorial Partner\n\n"}, {"role": "assistant", "content": "This is a TRAVEL piece by PureWow, Editorial Partner"}]}
{"messages": [{"role": "system", "content": "You are a expert news editor. You give author name and sort news headline into 5 categories: BUSINESS, POLITICS, TRAVEL, SPORTS, WELLNESS."}, {"role": "user", "content": "Headline: Elon Musk's Estranged Father Has Child With Stepdaughter, Says It's 'God's Plan'\n\n Author: Jenna Amatulli\n\n"}, {"role": "assistant", "content": "This is a BUSINESS piece by Jenna Amatulli"}]}


#### Fine-tuning model
- Upload file


In [45]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-1d0YWGGCxnWLsbGbXc4Gby90
Validation file ID: file-801xl69X9bSsT7Y4VqUqGjIX


- Create fine-tuning job

In [46]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner",
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])

Job ID: ftjob-8n7H3PvtiQz6h4V66n7Xryy0
Status: validating_files


- Check job status

In [48]:
response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])


Job ID: ftjob-8n7H3PvtiQz6h4V66n7Xryy0
Status: succeeded
Trained Tokens: 7136


- Check job running processes after completion

In [49]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 37/84: training loss=0.02, validation loss=0.00
Step 38/84: training loss=0.00, validation loss=0.10
Step 39/84: training loss=0.00, validation loss=0.00
Step 40/84: training loss=0.00, validation loss=0.00
Step 41/84: training loss=0.00, validation loss=0.00
Step 42/84: training loss=0.00, validation loss=0.00
Step 43/84: training loss=0.00, validation loss=0.00
Step 44/84: training loss=0.00, validation loss=0.00
Step 45/84: training loss=0.00, validation loss=0.00
Step 46/84: training loss=0.00, validation loss=0.00
Step 47/84: training loss=0.00, validation loss=0.00
Step 48/84: training loss=0.00, validation loss=0.01
Step 49/84: training loss=0.00, validation loss=0.00
Step 50/84: training loss=0.00, validation loss=0.00
Step 51/84: training loss=0.00, validation loss=0.00
Step 52/84: training loss=0.00, validation loss=0.00
Step 53/84: training loss=0.00, validation loss=0.00
Step 54/84: training loss=0.00, validation loss=0.00
Step 55/84: training loss=0.00, validation los

- Get fine-tuned model ID

In [50]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:kellogg-research-support:recipe-ner:8DzKAWX8


#### Using the new fine-tuned model

In [11]:
test_df = news_df.loc[31:40]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": create_user_message(test_row)})

pprint(test_messages)

[{'content': 'You are a expert news editor. You give author name and sort news '
             'headline into 5 categories: BUSINESS, POLITICS, TRAVEL, SPORTS, '
             'WELLNESS.',
  'role': 'system'},
 {'content': "Headline: 13 Hand Sanitizer Mistakes You're Probably Making\n"
             '\n'
             ' Author: Nicole Pajer\n'
             '\n',
  'role': 'user'}]


In [12]:
fine_tuned_model_id = "ft:gpt-3.5-turbo-0613:kellogg-research-support:recipe-ner:8DzKAWX8"
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=1500
)
print(response["choices"][0]["message"]["content"])
print("-----")
print(response)

This is a WELLNESS piece by Nicole Pajer
-----
{
  "id": "chatcmpl-8E0n57AifHLuXBOmcVtiZey2mQChE",
  "object": "chat.completion",
  "created": 1698350323,
  "model": "ft:gpt-3.5-turbo-0613:kellogg-research-support:recipe-ner:8DzKAWX8",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "This is a WELLNESS piece by Nicole Pajer"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 68,
    "completion_tokens": 11,
    "total_tokens": 79
  }
}


#### Original GPT-3.5-turbo & one-shot prompting
- GPT-3.5-turbo

In [13]:
### gpt-3.5-turbo
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=test_messages, temperature=0, max_tokens=1500
)
print(response["choices"][0]["message"]["content"])
print("-----")
print(response)

Category: WELLNESS
-----
{
  "id": "chatcmpl-8E0ocng3le3ad6Mkr7oqtONSxixEC",
  "object": "chat.completion",
  "created": 1698350418,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Category: WELLNESS"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 68,
    "completion_tokens": 4,
    "total_tokens": 72
  }
}


- One-shot prompting

In [15]:
# one-shot prompting
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {'content': 'You are a expert news editor. You give author name '
                    'and sort news headline into 5 categories: BUSINESS, '
                    'POLITICS, TRAVEL, SPORTS, WELLNESS.',
         'role': 'system'},
        {'content': 'Headline: Olympian Released From Beijing Isolation '
                    'Facility After Emotional Video Plea\n'
                    '\n'
                    ' Author: TIM REYNOLDS, AP\n'
                    '\n',
         'role': 'user'},
        {'content': 'This is a SPORTS piece by TIM REYNOLDS, AP',
         'role': 'assistant'},
        {'content': "Headline: 13 Hand Sanitizer Mistakes You're Probably Making\n\n Author: Nicole Pajer\n\n",
         'role': 'user'},
    ]
)
print(response["choices"][0]["message"]["content"])
print("-----")
print(response)

This is a WELLNESS article written by Nicole Pajer.
-----
{
  "id": "chatcmpl-8E0pF1dGwz0SWUak5O5MFddcZcHAZ",
  "object": "chat.completion",
  "created": 1698350457,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "This is a WELLNESS article written by Nicole Pajer."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 117,
    "completion_tokens": 13,
    "total_tokens": 130
  }
}
